In [1]:
#================================= Start of importing required packages and libraries =========================================#
from __future__ import print_function
%matplotlib inline
import numpy as np
import torch
from experiment_federated import *
import random
#================================== End of importing required packages and libraries ==========================================#

In [2]:
import torch
import subprocess

def get_free_gpu(threshold_mb=1000):
    """返回满足空闲显存条件的一块GPU ID，单位 MB"""
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_free = [int(x) for x in result.strip().split('\n')]
        for idx, mem in enumerate(memory_free):
            if mem > threshold_mb:
                return idx
    except Exception as e:
        print("无法检测GPU状态，默认使用CPU:", e)
    return None

free_gpu = get_free_gpu()

if free_gpu is not None and torch.cuda.is_available():
    DEVICE = torch.device(f'cuda:{free_gpu}')
    print(f"✅ 使用空闲GPU: cuda:{free_gpu}")
else:
    device = torch.device('cpu')
    print("⚠️ 没有可用GPU，改为使用CPU")


✅ 使用空闲GPU: cuda:0


In [3]:
#=============================== Defining global variables ========================#
DATASET_NAME = "PATHMNIST"
MODEL_NAME = "CNNPATH"
DD_TYPE = 'NON_IID'
ALPHA = 1
NUM_PEERS = 100 # "number of peers: K" 
FRAC_PEERS = 1 #'the fraction of peers: C to bel selected in each round'
SEED = 7 #fixed seed
random.seed(SEED)
CRITERION = nn.CrossEntropyLoss()
GLOBAL_ROUNDS = 200 #"number of rounds of federated model training"
LOCAL_EPOCHS = 3 #"the number of local epochs: E for each peer"
TEST_BATCH_SIZE = 1000
LOCAL_BS = 64 #"local batch size: B for each peer"
LOCAL_LR =  0.01#local learning rate: lr for each peer
LOCAL_MOMENTUM = 0.9 #local momentum for each peer
NUM_CLASSES = 9 # number of classes in an experiment
LABELS_DICT = {
    'Adipose': 0,
    'Background': 1,
    'Debris': 2,
    'Lymphocytes': 3,
    'Mucus': 4,
    'Smooth_muscle': 5,
    'Normal_colon_mucosa': 6,
    'Cancer_stroma': 7,
    'Adenocarcinoma_epithelium': 8
}
SOURCE_CLASS = 0 # the source class
TARGET_CLASS = 7 # the target class 
CLASS_PER_PEER = 10
SAMPLES_PER_CLASS = 582
RATE_UNBALANCE = 1

In [4]:
RULE = 'lfighter_dbo'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)


--> Starting experiment...
--> Loading of PATHMNIST dataset
--> Dataset has been loaded!
--> Creating CNNPATH model.....
--> Model has been created!
--> Distributing training data among peers
--> Training data have been distributed among peers
--> Creating peets instances
Data set: PATHMNIST
Data distribution: NON_IID
Aggregation rule: lfighter_dbo
Attack Type: label_flipping
Attackers Ratio: 40.0 %
Malicious Behavior Rate: 100 %

===>Simulation started...

====>Global model training started...



  0%|          | 0/200 [00:00<?, ?it/s]


 | Global training round : 1/200 |

[Debug] ⚠️ View 1 维度太大 (1384448), 执行 PCA 降维至 100
[Debug] ⚠️ View 2 维度太大 (150528), 执行 PCA 降维至 100
聚类 vs 实际攻击者标签:
[[24 36]
 [20 20]]
              precision    recall  f1-score   support

           0      0.545     0.400     0.462        60
           1      0.357     0.500     0.417        40

    accuracy                          0.440       100
   macro avg      0.451     0.450     0.439       100
weighted avg      0.470     0.440     0.444       100


Average test loss: 2.0245, Test accuracy: 2206/7180 (30.72%)

Class      - Accuracy
Adipose    - 0.0
Background - 67.3
Debris     - 33.6
Lymphocytes - 0.2
Mucus      - 0.0
Smooth_muscle - 52.4
Normal_colon_mucosa - 0.0
Cancer_stroma - 32.8
Adenocarcinoma_epithelium - 87.0

 | Global training round : 2/200 |



KeyboardInterrupt: 

In [ ]:
# Baseline|: FedAvg-no attacks (FL)
RULE = 'fedavg'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0, 0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'lfighter'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'median'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'tmean'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'mkrum'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'foolsgold'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'Tolpegin'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'FLAME'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)